In [ ]:
cd ~/Desktop/LinearProgramming
git commit
git push origin master

# Linear Programming (OPER 610) Project
## MAJ Matthew Ferguson

### Setup and test configuration
The first requirement is to import necessary methods from PuLP.
I've previously installed GNU LPK and the CBC Solver comes with PuLP. Gurobi is pending... hopefully that will be number three.  

the first block of code tests for connectivity of the Solvers.  Ideally, the output should appear something like:

Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver pulp.solvers.PULP_CBC_CMD passed.
Solver pulp.solvers.CPLEX_DLL unavailable
Solver pulp.solvers.CPLEX_CMD unavailable
Solver pulp.solvers.CPLEX_PY unavailable
Solver pulp.solvers.COIN_CMD unavailable
Solver pulp.solvers.COINMP_DLL unavailable
	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver pulp.solvers.GLPK_CMD passed.
Solver pulp.solvers.XPRESS unavailable
Solver pulp.solvers.GUROBI unavailable
Solver pulp.solvers.GUROBI_CMD unavailable
Solver pulp.solvers.PYGLPK unavailable
Solver pulp.solvers.YAPOSIB unavailable

In [23]:
from pulp import *

pulp.pulpTestAll()

	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver pulp.solvers.PULP_CBC_CMD passed.
Solver pulp.solvers.CPLEX_DLL unavailable
Solver pul

PulpError: Tests Failed

# Toy Model Instance

In [1]:
from pulp import *
import timeit

# Decision variables
x = [0, LpVariable("x1", 0, 100), LpVariable("x2", 0, 100)]
# 0<= x1 <= 100 & 0<= x2 <= 100
 
# Setup Problem -- are LpMax and LpMin statics for 0,1 or -1,1?
p = LpProblem("problem", LpMaximize)
 
# Set up matrix A and vector b
p += 2*x[1]+x[2] <= 150
p += x[1]+x[2] <= 100
p += x[1]<=50
#strictly necessary?
p += x[1]>=0
p += x[2]>=0
 
# ObjFunc
p += 3*x[1]+2*x[2]

Solvr = 0    
time = [0]

Solvr += 1
GNULPK = Solvr

# Pass to GNU Solver
start = timeit.default_timer()
status = p.solve(GLPK(msg=0))
time.append(timeit.default_timer() - start)
print(LpStatus[status])


Solvr +=1
CBC = Solvr
#CBC
start = timeit.default_timer()
status = p.solve()
time.append(timeit.default_timer() - start)
print(LpStatus[status])

Solvr += 1
GUR = Solvr

#CBC
start = timeit.default_timer()
status = p.solve(CPLEX(msg=0))
time.append(timeit.default_timer() - start)
print(LpStatus[status])



#GUROBI
# Pull license with grbgetkey from an academic ip address
# call for second test license

print("(x1,x2) is optimal at (" + str(value(x[1])) + ", "+ str(value(x[2])) + ")" )
print(str(time[1]) + " seconds of CPU time (GLPK).")
print(str(time[2]) + " seconds of CPU time (COIN/CBC).")
print(str(time[3]) + " seconds of CPU time (CPLEX).")


Optimal
Optimal
Optimal
(x1,x2) is optimal at (50.0, 50.0)
0.00716996192932 seconds of CPU time (GLPK).
0.00944709777832 seconds of CPU time (COIN/CBC).
0.00285410881042 seconds of CPU time (CPLEX).


### Now trying the pyomo package

In [12]:
import pyomo

s = open('pyomoTest.py', 'a+')

s.write('from pyomo.environ import * \n')
#just in-case we use integer values
s.write('from __future__ import division \n\n')

from pyomo.environ import *
from __future__ import division

s.write('p = AbstractModel()\n\n')
p = AbstractModel()

s.write('p.m = Param(within=NonNegativeIntegers)\n')
s.write('p.n = Param(within=NonNegativeIntegers)\n\n')
p.m = Param(within=NonNegativeIntegers)
p.n = Param(within=NonNegativeIntegers)

s.write('p.I = Set()\n')
s.write('p.J = Set()\n\n')
p.I = Set()
p.J = Set()

s.write('p.a = Param(p.I, p.J)\np.b = Param(p.I)\np.c = Param(p.J)\n\n')
p.a = Param(p.I, p.J)
p.b = Param(p.I)
p.c = Param(p.J)

s.write('p.x = Var(p.J, domain=NonNegativeReals)\n')
p.x = Var(p.J, domain=NonNegativeReals)

s.write('def obj_function(p):\n\treturn summation(p.c, p.a)\n\n')

def obj_function(p):
    return summation(p.c, p.a) #objfunc = sum of c_i * x_i over i

s.write('p.OBJ = Objective(rule=obj_function, sense=maximize)\n')
p.OBJ = Objective(rule=obj_function, sense=maximize)

s.write('def const_expression(p, i):\n\treturn sum(p.a[i,j] * p.x[j] for i in p.J) <= p.b[i]\n\n')
def const_expression(p, i):
    return sum(p.a[i,j] * p.x[j] for j in p.J) <= p.b[i]

s.write('p.AxbConstraint = Constraint(p.I, rule=const_expression)\n')
p.AxbConstraint = Constraint(p.I, rule=const_expression)

s.close()


In [16]:

s = open('pyomoTest.dat', 'a+')

s.write('set I := 1 2 3 ;\nset J := 1 2 ;\n\n')
s.write('param a :=\n1 1 2\n1 2 1\n2 1 1\n 2 2 1\n3 1 1\n3 2 0\n;\n\n')
     
s.write('param c :=\n1 3\n2 2\n;\n\n')
s.write('param b :=\n1 150\n2 100\n3 50\n;\n\n')

s.close()

hi


In [22]:
from pyomo.environ import * 
from __future__ import division 

p = AbstractModel()

p.m = Param(within=NonNegativeIntegers)
p.n = Param(within=NonNegativeIntegers)

p.I = Set()
p.J = Set()

p.a = Param(p.I, p.J)
p.b = Param(p.I)
p.c = Param(p.J)

p.x = Var(p.J, domain=NonNegativeReals)
def obj_function(p):
	return summation(p.c, p.a)

p.OBJ = Objective(rule=obj_function, sense=maximize)
def const_expression(p, i):
	return sum(p.a[i,j] * p.x[j] for i in p.J) <= p.b[i]

p.AxbConstraint = Constraint(p.I, rule=const_expression)



##---------------------------------------------------
##---------------------------------------------------

from pyomo.opt import SolverFactory
opt = SolverFactory('glpk')

problem = p.create_instance()
result = opt.solve(problem)

print(result)


ERROR:pyomo.core:Rule failed when generating expression for objective OBJ:
ValueError: Error executing summation(): The last argument value must be a variable or expression object if no 'index' option is specified
ERROR:pyomo.core:Constructing component 'OBJ' from data=None failed:
ValueError: Error executing summation(): The last argument value must be a variable or expression object if no 'index' option is specified


ERROR: Rule failed when generating expression for objective OBJ:
	ValueError: Error executing summation(): The last argument value must be a variable or expression object if no 'index' option is specified
ERROR: Constructing component 'OBJ' from data=None failed:
	ValueError: Error executing summation(): The last argument value must be a variable or expression object if no 'index' option is specified


ValueError: Error executing summation(): The last argument value must be a variable or expression object if no 'index' option is specified

In [ ]:
pyomo solve pyomoTest.py pyomoTest.dat

## Pyomo kind of sucks... can't script easily

Build out of Concrete Model sucks for scripting
Abstract model with AMPL input can work, but
    AbstractModel() is touchy...  getting error raised by parser with pyomo_create_model
    
    
